<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pathlib

In [2]:
file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

sheets_info = {
    'n_uni': {'skiprows': 2, 'cols': ['uni']},
    'n_proj': {'skiprows': 3, 'cols': ['proj', 'nome_proj']},
    'n_elem': {'skiprows': 3, 'cols': ['elem', 'nome_elem']},
    'n_vinc': {'skiprows': 3, 'cols': ['vinc', 'nome_vinc']},
    'n_rub': {'skiprows': 3, 'cols': ['rub', 'nome_rub']}
}

dfs = {
    sheet: pd.read_excel(file_path, sheet_name=sheet, skiprows=info['skiprows'], header=None, names=info['cols'])
    for sheet, info in sheets_info.items()
}

for sheet, df in dfs.items():
  if 'uni' in df.columns:
      df['nome_uni'] = df['uni'].str[5:]
      df['uni_int'] = df['uni'].str[:4].astype(int)
  if 'proj' in df.columns:
      df['proj_int'] = df['proj'].str[:4].astype(int)
  if 'elem' in df.columns:
      df['elem_int'] = df['elem'].str[:6].astype(int)
  if 'rub' in df.columns:
      df['rub_int'] = df['rub'].str[:12].astype('int64')
  if 'vinc' in df.columns:
      df['vinc_int'] = df['vinc'].str[:4].astype(int)

df_uni, df_proj, df_elem, df_vinc, df_rub = dfs.values()

In [3]:
# Generating a dataframe with paid expenses data from 2002 to 2023

# Define the folder path
exp_folder_2002_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2002_2023')

# Read all excel files efficiently
df_list = [
    pd.read_excel(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2002_2023.iterdir()
]

# Concatenate all DataFrames
df_exp_2002_2023 = pd.concat(df_list, ignore_index=True)

# Rename and clean up columns
df_exp_2002_2023 = df_exp_2002_2023[['CPTPAG', 'RESUL_PAGO', 'UNIDORC', 'PROJATIV', 'RUBRICA', 'VINCORC']].rename(columns={
    'CPTPAG':'Comp.pagto.', 'RESUL_PAGO':'Result. pago', 'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ',
    'RUBRICA':'Rubrica', 'VINCORC':'Vinc. Orçam.'
})

# Filter out rows with nan values in some columns
df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.', 'Result. pago'], ignore_index=True)

# Process categorical column efficiently
df_exp_2002_2023['Elemento'] = df_exp_2002_2023['Rubrica'].astype(str).str[:6].astype(int)

In [4]:
# Generating a dataframe with paid expenses data from 2024 onwards

# Define the folder path
exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

# Read all CSV files efficiently
df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.glob("*.csv")
]

# Concatenate all DataFrames
df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

# Convert financial columns efficiently
cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
df_exp_2024_onwards[cols_to_convert] = df_exp_2024_onwards[cols_to_convert].apply(
    lambda col: pd.to_numeric(col.str.replace('R\$ ', '', regex=True).str.replace(',', '.'), errors='coerce')
)

# Filter out rows with zero values in all columns to convert
df_exp_2024_onwards = df_exp_2024_onwards[
    df_exp_2024_onwards[cols_to_convert].ne(0).any(axis=1)
]

# Rename and clean up columns
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
}, inplace=True)

# Fix month values
df_exp_2024_onwards['MONTH'] = df_exp_2024_onwards['MONTH'].replace({0: 1, 13: 12}).astype(int)

# Convert date
df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

# Process categorical columns efficiently
df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)

# Sorting once before processing
df_exp_2024_onwards.sort_values('Comp.pagto.', inplace=True, ignore_index=True)

# Subtract the last nonzero appearance for target columns
cols = ['Proj/Ativ', 'Rubrica', 'fonterecursos', 'YEAR']
target_cols = ['restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
for target_col in target_cols:
    df_exp_2024_onwards[target_col + '_prev'] = (
        df_exp_2024_onwards.where(df_exp_2024_onwards[target_col] != 0).groupby(cols)[target_col].shift(1).fillna(0)
    )
    df_exp_2024_onwards[target_col] -= df_exp_2024_onwards[target_col + '_prev']

# Compute final results
df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1)

# Final filter and reordering columns
df_exp_2024_onwards = df_exp_2024_onwards[df_exp_2024_onwards['Result. pago'] != 0].reset_index(drop=True)
df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)

df_exp_2024_onwards = df_exp_2024_onwards[
    ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
     'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos']
]

In [5]:
# concatenating the expense dataframes
relevant_columns = ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
df_exp = pd.concat([df_exp_2002_2023, df_exp_2024_onwards[relevant_columns]], ignore_index=True)

In [6]:
# Updating the elements dataframe

df_new_elem = df_exp_2024_onwards[['Elemento', 'deelemento']].drop_duplicates(keep='last').rename(columns={
    'Elemento': 'elem_int',
    'deelemento': 'nome_elem'
})
df_elem = pd.concat([df_elem, df_new_elem], ignore_index=True).drop_duplicates(subset='elem_int', keep='last').reset_index(drop=True)
df_elem['elem'] = df_elem['elem_int'].astype(str) + ' - ' + df_elem['nome_elem'].str.title()

In [7]:
# Updating the rubric dataframe

df_new_rub = df_exp_2024_onwards[['Rubrica', 'dedesdobramento']].drop_duplicates(keep='last').rename(columns={
    'Rubrica': 'rub_int',
    'dedesdobramento': 'nome_rub'
})
df_rub = pd.concat([df_rub, df_new_rub], ignore_index=True).drop_duplicates(subset='rub_int', keep='last').reset_index(drop=True)
df_rub['rub'] = df_rub['rub_int'].astype(str) + ' - ' + df_rub['nome_rub'].str.title()

In [8]:
# Updating the project dataframe

df_new_proj = df_exp_2024_onwards[['Proj/Ativ', 'desubacao']].drop_duplicates(keep='last').rename(columns={
    'Proj/Ativ': 'proj_int',
    'desubacao': 'nome_proj'
})
df_proj = pd.concat([df_proj, df_new_proj], ignore_index=True).drop_duplicates(subset='proj_int', keep='last').reset_index(drop=True)
df_proj['proj'] = df_proj['proj_int'].astype(str) + ' - ' + df_proj['nome_proj'].str.title()

In [9]:
# Updating the budget units dataframe

df_new_uni = df_exp_2024_onwards[['Unid. Orçam.', 'deunidadeorcamentária']].drop_duplicates(keep='last').rename(columns={
    'Unid. Orçam.': 'uni_int',
    'deunidadeorcamentária': 'nome_uni'
})
df_uni = pd.concat([df_uni, df_new_uni], ignore_index=True).drop_duplicates(subset='uni_int', keep='last').reset_index(drop=True)
df_uni['uni'] = df_uni['uni_int'].astype(str) + ' - ' + df_uni['nome_uni'].str.title()

In [10]:
# Updating the entail dataframe

df_new_vinc = df_exp_2024_onwards[['Vinc. Orçam.', 'defonterecursos']].drop_duplicates(keep='last').rename(columns={
    'Vinc. Orçam.': 'vinc_int', 'defonterecursos': 'nome_vinc'
})
df_new_vinc['nome_vinc'] = df_new_vinc['nome_vinc'].str.split('-').str.get(1).str.strip()
df_vinc = pd.concat([df_vinc, df_new_vinc], ignore_index=True).drop_duplicates(subset='vinc_int', keep='last').reset_index(drop=True)
df_vinc['vinc'] = df_vinc['vinc_int'].astype(str) + ' - ' + df_vinc['nome_vinc'].str.title()

In [ ]:
final_data_folder = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/final_data')

df_uni.sort_values('uni_int').to_csv(final_data_folder / 'df_uni.csv', encoding='utf-8-sig', sep=';', index=False)
df_proj.sort_values('proj_int').to_csv(final_data_folder / 'df_proj.csv', encoding='utf-8-sig', sep=';', index=False)
df_elem.sort_values('elem_int').to_csv(final_data_folder / 'df_elem.csv', encoding='utf-8-sig', sep=';', index=False)
df_rub.sort_values('rub_int').to_csv(final_data_folder / 'df_rub.csv', encoding='utf-8-sig', sep=';', index=False)
df_vinc.sort_values('vinc_int').to_csv(final_data_folder / 'df_vinc.csv', encoding='utf-8-sig', sep=';', index=False)
df_exp.sort_values('Comp.pagto.').to_csv(final_data_folder / 'df_exp.csv', encoding='utf-8-sig', sep=';', index=False)

**Revenue**

In [31]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                                 'Mês': 'MONTH',
                                 'Rótulos de Linha': 'orgao',
                                 'Vínculo Cod': 'vinculo',
                                 'N6 Subalinea': 'desdobramento6',
                                 'Valor Arrecadado': 'valor_arrecadado',
                                 'Valor Orçado': 'valor_orcado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
# df_rev_2004_2017['vinculo'] = df_rev_2004_2017['vinculo'].astype('str').replace('400', '0400').replace('6051', '605X')\
#   .replace('1', '0001').replace('605X', '6051')

In [49]:
# Generating a dataframe with revenue collected from 2018 to 2023

rev_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_list = [
    pd.read_excel(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2018_2023.iterdir()
]

df_rev_2018_2023 = pd.concat(df_list, ignore_index=True)

df_rev_2018_2023.drop(columns=['valor_cancelado', 'valor_lancado', 'valor_meta', 'informacao_complementar'], inplace=True)
df_rev_2018_2023 = df_rev_2018_2023.loc[df_rev_2018_2023['orgao_raiz'] == 7000].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023.sort_values('Data', inplace=True, ignore_index=True)
# df_rev_2018_2023['vinculo'] = df_rev_2018_2023['vinculo'].astype(str).replace({'400': '0400'})
# df_rev_2018_2023['rubrica'] = df_rev_2018_2023['rubrica'].astype(str)

# df_rev_2018_2023['Natureza da Receita Antiga'] = df_rev_2018_2023['rubrica'].str.pad(14, fillchar='0').str.replace(
#     r'(\d{2})(\d)(\d)(\d)(\d{2})(\d)(\d)(\d{2})(\d{2})(\d+)', r'\1.\2.\3.\4.\5.\6.\7.\8.\9.\10', regex=True
# ) + '.' + df_rev_2018_2023['vinculo'] + '.0'

In [43]:
df_rev_2018_2023.columns

Index(['YEAR', 'MONTH', 'orgao_raiz', 'nome_orgao', 'orgao', 'nome_orgao.1',
       'rubrica', 'vinculo', 'digito', 'categoria', 'origem', 'especie',
       'desdobramento1', 'desdobramento2', 'desdobramento3', 'tipo',
       'desdobramento4', 'desdobramento5', 'desdobramento6',
       'valor_arrecadado', 'valor_orcado', 'Data'],
      dtype='object')

In [46]:
df_rev_2018_2023

,YEAR,MONTH,orgao_raiz,nome_orgao,orgao,nome_orgao.1,rubrica,vinculo,digito,categoria,...,desdobramento1,desdobramento2,desdobramento3,tipo,desdobramento4,desdobramento5,desdobramento6,valor_arrecadado,valor_orcado,Data
0,2018,12,7000,PREVIMPA E FUNDOS,7201,7201 - FUNDO DE REAPARELHAMENTO E APERFEIÇOAM...,13210041010200,6067,0,1 - Receitas Correntes,...,1321 - Juros e Correções Monetárias,132100,1321004 - Rem. Recursos do Reg. Próprio de Pre...,13210041 - Rem. dos Rec. do Reg. Próprio de Pr...,1321004101 - Remuneração de Recursos Aplicados...,132100410102 - Fundos de Investimentos em Rend...,13210041010200,43039.58,461407.0,2018-12-01
1,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100412027100,6050,0,7 - Receitas Correntes Intra-Orçamentárias,...,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120271,72100412027100,0.00,0.0,2018-12-01
2,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100412027100,6069,0,7 - Receitas Correntes Intra-Orçamentárias,...,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120271,72100412027100,0.00,0.0,2018-12-01
3,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100422011100,400,0,7 - Receitas Correntes Intra-Orçamentárias,...,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210042,72100422 - Contribuição do Servidor Ativo Civi...,7210042201 - Contr. do Serv. At. Civil p/ RPPS...,721004220111,72100422011100,0.00,0.0,2018-12-01
4,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100422023100,6050,0,7 - Receitas Correntes Intra-Orçamentárias,...,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210042,72100422 - Contribuição do Servidor Ativo Civi...,7210042202 - Contrib. dos Servidores - At. Civ...,721004220231,72100422023100,0.00,0.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18069,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180121020171,6050,0,1 - Receitas Correntes,...,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12180121020171 - CPSSS do Servidor Civil Inati...,627.64,0.0,2020-01-01
18070,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180121020161,6050,0,1 - Receitas Correntes,...,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12180121020161 - CPSSS do Servidor Civil Inati...,1141.11,0.0,2020-01-01
18071,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180111030101,400,0,1 - Receitas Correntes,...,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218011 - CPSSS do Servidor Civil Ativo,12180111 - CPSSS do Servidor Civil Ativo - Pri...,1218011103 - CPSSS do Servidor Civil Ativo - P...,121801110301 - CPSSS do Servidor Civil Ativo -...,12180111030101 - CPSSS do Servidor Civil Ativo...,3405.55,0.0,2020-01-01
18072,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180111010141,400,0,1 - Receitas Correntes,...,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. 

In [58]:
df_rev_2018_2023[['rubrica', 'desdobramento1', 'desdobramento2', 'desdobramento3', 'tipo', 'desdobramento4', 'desdobramento5', 'desdobramento6']].drop_duplicates(subset='rubrica', keep='last')

,rubrica,desdobramento1,desdobramento2,desdobramento3,tipo,desdobramento4,desdobramento5,desdobramento6
2262,72180121015100,7218 - Contribuições Sociais Específicas de Es...,721801 - Contribuições para o RPPS de Estados/...,7218012 - Contribuição Patronal dos Servidores...,72180121 - Contribuição Patronal dos Servidore...,7218012101 - Contribuição Patronal dos Servido...,721801210151,72180121015100
2264,12100423030100,1210,121004 - Contribuição para o Regime Próprio de...,1210042 - Contribuição do Servidor Ativo Civil...,12100423 - Contribuição do Servidor Ativo Civi...,1210042303 - Contribuição do Servidor Afastado...,121004230301 - Contribuição do Servidor Afasta...,12100423030100
2266,13290011000000,1329 - Outros Valores Mobiliários,132900,1329001 - Outros Valores Mobiliários,13290011 - Outros Valores Mobiliários - Principal,1329001100,132900110000,13290011000000
2268,19909911019800,1990,199099 - Outras Receitas,1990991 - Outras Receitas - Primárias,19909911 - Outras Receitas - Primárias - Princ...,1990991101 - Outras Receitas Diretamente Arrec...,199099110198,19909911019800
2269,72100412025100,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120251,72100412025100
...,...,...,...,...,...,...,...,...
18069,72150211010111,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,721502 - Contribuição Patronal - Servidor Civi...,7215021 - Contr. Patronal - Servidor Civil Ati...,72150211 - Contr. Patronal - Servidor Civil At...,7215021101 - Contr. Patr. - Servidor Civil Ati...,721502110101 - Contr. Patronal - Servidor Civi...,72150211010111 - Contr. Patronal - Servidor At...
18070,12150121010131,1215 - Contrib. para Regimes Próprios de Prev...,121501 - Contribuição do Servidor Civil,1215012 - Contr.do Servidor Civil Inativo,12150121 - Contr.do Servidor Civil Inativo - P...,1215012101 - Contr.do Servidor Civil Inativo -...,121501210101 - Contr.do Servidor Civil Inativo...,12150121010131 - Contr.do Servidor Civil Inati...
18071,12150214040201,1215 - Contrib. para Regimes Próprios de Prev...,121502 - Contribuição Patronal - Servidor Civil,1215021 - Contr.Patronal - Servidor Civil Ativo,12150214 - Contr.Patronal - Servidor Civil Ati...,1215021404 - Contr.Patronal - Servidores Cedid...,121502140402 - Contr.Patr.- Cedidos- Dív.Ativa...,12150214040201 - Contr.Patr.-Cedidos-Dív.Ativa...
18072,12150121010141,1215 - Contrib. para Regimes Próprios de Prev...,121501 - Contribuição do Servidor Civil,1215012 - Contr.do Servidor Civil Inativo,12150121 - Contr.do Servidor Civil Inativo - P...,1215012101 - Contr.do Servidor Civil Inativo -...,121501210101 - Contr.do Servidor Civil Inativo...,12150121010141 - Contr.do Servidor Civil Inati...


In [67]:
len(df_rev_2018_2023['rubrica'].unique())

587

In [14]:
# Generating a dataframe with revenue collected from 2024 onwards

rev_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2024+')

df_list = [
    pd.read_csv(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2024_onwards.iterdir()
]

df_rev_2024_onwards = pd.concat(df_list, ignore_index=True)

df_rev_2024_onwards.rename(columns={'exercicio': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2024_onwards['Data'] = pd.to_datetime(df_rev_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

In [83]:
df_rev_2004_2017

,orgao,vinculo,desdobramento6,YEAR,MONTH,valor_arrecadado,valor_orcado,Data
1,7000,1,19122902 - Multas e Juros de Mora das Contribu...,2007,1,0.00,0.00,2007-01-01
2,7000,1,19122902 - Multas e Juros de Mora das Contribu...,2007,2,0.00,0.00,2007-02-01
3,7000,1,19122902 - Multas e Juros de Mora das Contribu...,2007,3,42.41,0.00,2007-03-01
4,7000,1,19122902 - Multas e Juros de Mora das Contribu...,2007,4,0.00,0.00,2007-04-01
5,7000,1,19122902 - Multas e Juros de Mora das Contribu...,2007,5,0.00,0.00,2007-05-01
...,...,...,...,...,...,...,...,...
9404,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,8,61393.81,17841.66,2017-08-01
9405,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,9,57236.33,17841.67,2017-09-01
9406,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,10,62636.51,17841.66,2017-10-01
9407,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,11,58037.88,17841.68,2017-11-01


In [34]:
df_rev_2004_2017[['desdobramento6', 'vinculo']].drop_duplicates()

,desdobramento6,vinculo
1,19122902 - Multas e Juros de Mora das Contribu...,1
31,91328200 - (-)deduções de Remuneração dos Inve...,1
51,12102901 - Contribuição Patronal Para o Regime...,400
161,12102907 - Contribuição de Servidor - Ativo Civil,400
329,12102909 - Contribuição de Servidor - Inativo ...,400
...,...,...
9352,16001307 - Serviços de Cópias Xerográficas E/o...,6069
9354,19229900 - Outras Restituições,6069
9365,19909800 - Descontos Obtidos,6069
9367,72102901 - Contribuição Patronal de Servidor A...,6069


In [86]:
df_rev_2004_2017['desdobramento6'].unique()

array(['19122902 - Multas e Juros de Mora das Contribuições do Servidor',
       '91328200 - (-)deduções de Remuneração dos Investimentos Em Renda Variavel',
       '12102901 - Contribuição Patronal Para o Regime Próprio de Previdencia',
       '12102907 - Contribuição de Servidor - Ativo Civil',
       '12102909 - Contribuição de Servidor - Inativo Civil',
       '12102911 - Contribuição de Pensionista Civil',
       '12102999 - Outras Contribuições Previdenciárias',
       '12104600 - Compensação Previdenciária Entre Rgps e Os Rpps',
       '12109900 - Outras Contribuições Sociais',
       '13210601 - Títulos de Responsabilidade do Governo Federal-vinculados Ao Rpps',
       '13219900 - Outros Rendimentos de Títulos',
       '13240100 - Fundos de Investimentos Renda Fixa',
       '13240200 - Fundos de Aplicações Em Cotas-renda Fixa',
       '13249900 - Outros Fundos de Investimentos',
       '13250199 - Receita de Remuneração De Outros Depósitos Bancários Rec.vinculados',
       '132

In [16]:
df_rev_2004_2017

,orgao,vinculo,desdobramento6,YEAR,MONTH,valor_arrecadado,valor_orcado,Data
1,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,1,0.00,0.00,2007-01-01
2,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,2,0.00,0.00,2007-02-01
3,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,3,42.41,0.00,2007-03-01
4,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,4,0.00,0.00,2007-04-01
5,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,5,0.00,0.00,2007-05-01
...,...,...,...,...,...,...,...,...
9404,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,8,61393.81,17841.66,2017-08-01
9405,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,9,57236.33,17841.67,2017-09-01
9406,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,10,62636.51,17841.66,2017-10-01
9407,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,11,58037.88,17841.68,2017-11-01


In [17]:
df_rev_2018_2023

,YEAR,MONTH,orgao_raiz,nome_orgao,orgao,nome_orgao.1,rubrica,vinculo,digito,categoria,...,desdobramento2,desdobramento3,tipo,desdobramento4,desdobramento5,desdobramento6,valor_arrecadado,valor_orcado,Data,Natureza da Receita Antiga
0,2018,12,7000,PREVIMPA E FUNDOS,7201,7201 - FUNDO DE REAPARELHAMENTO E APERFEIÇOAM...,13210041010200,6067,0,1 - Receitas Correntes,...,132100,1321004 - Rem. Recursos do Reg. Próprio de Pre...,13210041 - Rem. dos Rec. do Reg. Próprio de Pr...,1321004101 - Remuneração de Recursos Aplicados...,132100410102 - Fundos de Investimentos em Rend...,13210041010200,43039.58,461407.0,2018-12-01,13.2.1.0.04.1.0.10.20.0.6067.0
1,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100412027100,6050,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120271,72100412027100,0.00,0.0,2018-12-01,72.1.0.0.41.2.0.27.10.0.6050.0
2,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100412027100,6069,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120271,72100412027100,0.00,0.0,2018-12-01,72.1.0.0.41.2.0.27.10.0.6069.0
3,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100422011100,0400,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210042,72100422 - Contribuição do Servidor Ativo Civi...,7210042201 - Contr. do Serv. At. Civil p/ RPPS...,721004220111,72100422011100,0.00,0.0,2018-12-01,72.1.0.0.42.2.0.11.10.0.0400.0
4,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100422023100,6050,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210042,72100422 - Contribuição do Servidor Ativo Civi...,7210042202 - Contrib. dos Servidores - At. Civ...,721004220231,72100422023100,0.00,0.0,2018-12-01,72.1.0.0.42.2.0.23.10.0.6050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18069,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180121020171,6050,0,1 - Receitas Correntes,...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12180121020171 - CPSSS do Servidor Civil Inati...,627.64,0.0,2020-01-01,12.1.8.0.12.1.0.20.17.1.6050.0
18070,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180121020161,6050,0,1 - Receitas Correntes,...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12180121020161 - CPSSS do Servidor Civil Inati...,1141.11,0.0,2020-01-01,12.1.8.0.12.1.0.20.16.1.6050.0
18071,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180111030101,0400,0,1 - Receitas Correntes,...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218011 - CPSSS do Servidor Civil Ativo,12180111 - CPSSS do Servidor Civil Ativo - Pri...,1218011103 - CPSSS do Servidor Civil Ativo - P...,121801110301 - CPSSS do Servidor Civil Ativo -...,12180111030101 - CPSSS do Servidor Civil Ativo...,3405.55,0.0,2020-01-01,12.1.8.0.11.1.0.30.10.1.0400.0
18072,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180111010141,0400,0,1 - Receitas Correntes,...,121801 - Contr.

In [18]:
df_rev_2024_onwards

,YEAR,MONTH,orgao,unidadegestora,gestao,unidadeorcamentaria,categoriaeconomica,origem,especie,d1,...,valorarrecadadobruto,valoratualizadoliquido,valordeducao,valordeducaoatualizado,valordeducaoorcado,valororcadoliquido,valorrealizadoliquido,valorreceitaatualizado,valorreceitaorcado,Data
0,2024,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707001 - ADMINISTRAÇÃO PREVIMPA,00003 - GESTÃO PRÓPRIA,07001 - ADMINISTRAÇÃO PREVIMPA,1 - Receitas Correntes,13 - Receita Patrimonial,132 - Valores Mobiliários,1321 - Juros e Correções Monetárias,...,NaN,2044275.0,NaN,0.0,0.0,2044275.0,NaN,2044275.0,2044275.0,2024-01-01
1,2024,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707001 - ADMINISTRAÇÃO PREVIMPA,00003 - GESTÃO PRÓPRIA,07001 - ADMINISTRAÇÃO PREVIMPA,1 - Receitas Correntes,13 - Receita Patrimonial,132 - Valores Mobiliários,1321 - Juros e Correções Monetárias,...,138006.05,NaN,NaN,NaN,NaN,NaN,138006.05,NaN,NaN,2024-01-01
2,2024,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707001 - ADMINISTRAÇÃO PREVIMPA,00003 - GESTÃO PRÓPRIA,07001 - ADMINISTRAÇÃO PREVIMPA,1 - Receitas Correntes,16 - Receita de Serviços,161 - Serviços Administrativos e Comerciais Ge...,1611 - Serviços Administrativos e Comerciais G...,...,NaN,1440.0,NaN,0.0,0.0,1440.0,NaN,1440.0,1440.0,2024-01-01
3,2024,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707001 - ADMINISTRAÇÃO PREVIMPA,00003 - GESTÃO PRÓPRIA,07001 - ADMINISTRAÇÃO PREVIMPA,1 - Receitas Correntes,19 - Outras Receitas Correntes,"192 - Indenizações, Restituições e Ressarcimentos",1922 - Restituições,...,NaN,1651.0,NaN,0.0,0.0,1651.0,NaN,1651.0,1651.0,2024-01-01
4,2024,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707001 - ADMINISTRAÇÃO PREVIMPA,00003 - GESTÃO PRÓPRIA,07001 - ADMINISTRAÇÃO PREVIMPA,1 - Receitas Correntes,19 - Outras Receitas Correntes,"192 - Indenizações, Restituições e Ressarcimentos",1922 - Restituições,...,NaN,1694.0,NaN,0.0,0.0,1694.0,NaN,1694.0,1694.0,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648,2025,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707003 - REGIME DE CAPITALIZAÇÃO,00003 - GESTÃO PRÓPRIA,07003 - REGIME DE CAPITALIZAÇÃO,7 - Receitas Correntes Intra-Orçamentárias,72 - Contribuições Intra-Orçamentárias,721 - Contribuições Sociais Intra-Orçamentárias,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,...,NaN,5969.0,NaN,0.0,0.0,5969.0,NaN,5969.0,5969.0,2025-01-01
1649,2025,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707003 - REGIME DE CAPITALIZAÇÃO,00003 - GESTÃO PRÓPRIA,07003 - REGIME DE CAPITALIZAÇÃO,7 - Receitas Correntes Intra-Orçamentárias,72 - Contribuições Intra-Orçamentárias,721 - Contribuições Sociais Intra-Orçamentárias,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,...,460.79,NaN,NaN,NaN,NaN,NaN,460.79,NaN,NaN,2025-01-01
1650,2025,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707003 - REGIME DE CAPITALIZAÇÃO,00003 - GESTÃO PRÓPRIA,07003 - REGIME DE CAPITALIZAÇÃO,7 - Receitas Correntes Intra-Orçamentárias,72 - Contribuições Intra-Orçamentárias,721 - Contribuições Sociais Intra-Orçamentárias,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,...,NaN,3857.0,NaN,0.0,0.0,3857.0,NaN,3857.0,3857.0,2025-01-01
1651,2025,1,07000 - DEPARTAMENTO MUNICIPAL DE PREVIDÊNCIA ...,707003 - REGIME DE CAPITALIZAÇÃO,00003 - GESTÃO PRÓPRIA,07003 - REGIME DE CAPITALIZAÇÃO,7 - Receitas Correntes Intra-Orçamentárias,72 - Contribuições Intra-Orçamentárias,721 - Contribuições Sociais Intra-Orçamentárias,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,...,301.76,NaN,NaN,NaN,NaN,NaN,301.76,NaN,NaN,2025-01-01


In [19]:
df_rev_2004_2017.columns

Index(['orgao', 'vinculo', 'desdobramento6', 'YEAR', 'MONTH',
       'valor_arrecadado', 'valor_orcado', 'Data'],
      dtype='object')

In [20]:
df_rev_2018_2023.columns

Index(['YEAR', 'MONTH', 'orgao_raiz', 'nome_orgao', 'orgao', 'nome_orgao.1',
       'rubrica', 'vinculo', 'digito', 'categoria', 'origem', 'especie',
       'desdobramento1', 'desdobramento2', 'desdobramento3', 'tipo',
       'desdobramento4', 'desdobramento5', 'desdobramento6',
       'valor_arrecadado', 'valor_orcado', 'Data',
       'Natureza da Receita Antiga'],
      dtype='object')

In [21]:
df_rev_2024_onwards.columns

Index(['YEAR', 'MONTH', 'orgao', 'unidadegestora', 'gestao',
       'unidadeorcamentaria', 'categoriaeconomica', 'origem', 'especie', 'd1',
       'dd2', 'd3', 'tipo', 'desdobramentosigef', 'rubricacompleta',
       'fonterecurso', 'codificacaotce', 'valorarrecadadobruto',
       'valoratualizadoliquido', 'valordeducao', 'valordeducaoatualizado',
       'valordeducaoorcado', 'valororcadoliquido', 'valorrealizadoliquido',
       'valorreceitaatualizado', 'valorreceitaorcado', 'Data'],
      dtype='object')

In [69]:
df_rev_2024_onwards[['origem', 'especie', 'd1', 'dd2', 'd3', 'tipo', 'desdobramentosigef', 'rubricacompleta']].drop_duplicates()

,origem,especie,d1,dd2,d3,tipo,desdobramentosigef,rubricacompleta
0,13 - Receita Patrimonial,132 - Valores Mobiliários,1321 - Juros e Correções Monetárias,132104 - Rem. Recursos do Reg. Próprio de Prev...,1321040 - Rem. Recursos do Reg. Próprio de Pre...,13210401 - Rem. dos Rec. do Reg. Próprio de Pr...,1321040102 - Fundos de Investimentos em Renda ...,1321040102 - Fundos de Investimentos em Renda ...
2,16 - Receita de Serviços,161 - Serviços Administrativos e Comerciais Ge...,1611 - Serviços Administrativos e Comerciais G...,161101 - Serv.Adm.e Comerc.Gerais Prestados po...,1611010 - Serv.Adm.e Comerc.Gerais Prestados p...,16110101 - Serv.Adm.e Com.Gerais Prest. Entida...,1611010110 - Serv. de Cópias Xerográf. e/ou Có...,1611010110 - Serv. de Cópias Xerográf. e/ou Có...
3,19 - Outras Receitas Correntes,"192 - Indenizações, Restituições e Ressarcimentos",1922 - Restituições,192299 - Outras Restituições,1922990 - Outras Restituições,19229901 - Outras Restituições - Principal,1922990157 - Sobra de Adiantamento de Numerári...,1922990157 - Sobra de Adiantamento de Numerári...
4,19 - Outras Receitas Correntes,"192 - Indenizações, Restituições e Ressarcimentos",1922 - Restituições,192299 - Outras Restituições,1922990 - Outras Restituições,19229901 - Outras Restituições - Principal,1922990161 - Restituições Diversas - Taxa de A...,1922990161 - Restituições Diversas - Taxa de A...
5,72 - Contribuições Intra-Orçamentárias,721 - Contribuições Sociais Intra-Orçamentárias,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,721502 - Contribuição Patronal - Servidor Civi...,7215021 - Contr. Patronal - Servidor Civil Ati...,72150211 - Contr. Patronal - Servidor Civil At...,7215021111 - Contr. Patronal - Servidor Ativo ...,7215021111 - Contr. Patronal - Servidor Ativo ...
...,...,...,...,...,...,...,...,...
1165,12 - Contribuições,121 - Contribuições Sociais,1215 - Contrib. para Regimes Próprios de Prev...,121502 - Contribuição Patronal - Servidor Civil,1215021 - Contr.Patronal - Servidor Civil Ativo,12150213 - Contr.Patronal - Servidor Civil Ati...,1215021391 - Contr.Patronal - Serv. Afastados ...,1215021391 - Contr.Patronal - Serv. Afastados ...
1166,12 - Contribuições,121 - Contribuições Sociais,1215 - Contrib. para Regimes Próprios de Prev...,121502 - Contribuição Patronal - Servidor Civil,1215021 - Contr.Patronal - Servidor Civil Ativo,12150214 - Contr.Patronal - Servidor Civil Ati...,1215021491 - Contr.Patronal - Afastados-Dív. A...,1215021491 - Contr.Patronal - Afastados-Dív. A...
1241,72 - Contribuições Intra-Orçamentárias,721 - Contribuições Sociais Intra-Orçamentárias,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,721551 - Contr.Patronal - Parcelamentos - Intr...,7215511 - Contr.Patronal - Parcelamentos - Ser...,72155112 - Contr.Patronal - Parcelamentos - Se...,7215511267 - Parcel. Déb. Patr. - Termo 682/18...,7215511267 - Parcel. Déb. Patr. - Termo 682/18...
1472,19 - Outras Receitas Correntes,"192 - Indenizações, Restituições e Ressarcimentos",1922 - Restituições,192299 - Outras Restituições,1922990 - Outras Restituições,19229901 - Outras Restituições - Principal,1922990156 - Restituições de Servidores - Taxa...,1922990156 - Restituições de Servidores - Taxa...


In [82]:
df_rev_2024_onwards['origem'].unique()

array(['13 - Receita Patrimonial', '16 - Receita de Serviços',
       '19 - Outras Receitas Correntes',
       '72 - Contribuições Intra-Orçamentárias',
       '73 - Receita Patrimonial Intra-Orçamentárias',
       '12 - Contribuições',
       '79 - Outras Receitas Correntes Intra-Orçamentárias'], dtype=object)

In [81]:
df_rev_2024_onwards['tipo'].unique()

array(['13210401 - Rem. dos Rec. do Reg. Próprio de Prev. Social - RPPS - Principal',
       '16110101 - Serv.Adm.e Com.Gerais Prest. Entidades e Órgãos Púb.em Geral-Principal',
       '19229901 - Outras Restituições - Principal',
       '72150211 - Contr. Patronal - Servidor Civil Ativo - Principal -Intra-Orçamentária',
       '73110111 - Aluguéis e Arrendamentos - Principal',
       '12150111 - Contr.do Servidor Civil Ativo - Principal',
       '12150112 - Contr.do Servidor Civil Ativo - Multas e Juros',
       '12150113 - Contr.do Servidor Civil Ativo - Dívida Ativa',
       '12150114 - Contr.do Servidor Civil Ativo - Dívida Ativa - Multas e Juros',
       '12150121 - Contr.do Servidor Civil Inativo - Principal',
       '12150131 - Contr.do Servidor Civil - Pensionistas - Principal',
       '12150211 - Contr.Patronal - Servidor Civil Ativo - Principal',
       '12150212 - Contr.Patronal - Servidor Civil Ativo - Multas e Juros',
       '12150213 - Contr.Patronal - Servidor Civil Ativ

In [72]:
df_rev_2024_onwards['rubricacompleta'].unique()

array(['1321040102 - Fundos de Investimentos em Renda Fixa',
       '1611010110 - Serv. de Cópias Xerográf. e/ou Cópias Heliográf. - Taxa de Adm. RPPS',
       '1922990157 - Sobra de Adiantamento de Numerário - Taxa de Administração do RPPS',
       '1922990161 - Restituições Diversas - Taxa de Administração do RPPS',
       '7215021111 - Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada',
       '7215021112 - Contr.Patronal - Servidor Ativo - Plano em Capitalização -Centralizada',
       '7215021121 - Contr. Patronal - Servidor Ativo - Plano em Repartição - CMPA',
       '7215021122 - Contr. Patronal - Servidor Ativo - Plano em Capitalização - CMPA',
       '7215021131 - Contr. Patronal - Servidor Ativo - Plano em Repartição - DEMHAB',
       '7215021132 - Contr. Patronal - Servidor Ativo - Plano em Capitalização - DEMHAB',
       '7215021141 - Contr. Patronal - Servidor Ativo - Plano em Repartição - DMAE',
       '7215021142 - Contr. Patronal - Servidor Ativo - Pl